# A priori statistics and correction

This notebook analyses the a priori statistics of the training data and calculates the correction ratios.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import xarray as xr
from hydronn.utils import load_style

load_style()

## Training data statistics

The code below iterates through the training data and calculates the precipitation distributions by month.

In [ ]:
from quantnn.qrnn import QRNN
qrnn = QRNN.load("/home/simonpf/src/hydronn/models/hydronn.pckl")
bins = qrnn.bins.copy()
qrnn = QRNN.load("/home/simonpf/src/hydronn/models/hydronn_4_all.pckl")
bins_4 = qrnn.bins.copy()

In [ ]:
from pathlib import Path
from hydronn.data.training_data import decompress_and_load
training_files = list(Path("/home/simonpf/data/hydronn/training_data/").glob("*nc*"))

month_bins = np.arange(0.5, 12.6)
counts = np.zeros((12, bins.size - 1))

for filename in training_files:
    data = decompress_and_load(filename)

    sp = data.surface_precip.data.copy()
    m = np.tile(data.time_goes.dt.month.data.reshape(-1, 1, 1), (1,) + sp.shape[1:])
    # Replace zeros as is done for training.
    indices = sp < 1e-2
    sp[indices] = 10 ** np.random.uniform(-4, -2, size=indices.sum())
    sp = np.maximum(sp, 1e-3)
    cts, _, _ = np.histogram2d(m.ravel(), sp.ravel(), bins=(month_bins, bins))
    counts += cts


In [ ]:
gauges = xr.load_dataset("/home/simonpf/data/hydronn/gauge_data/data.nc")
start_date = np.datetime64("2019-01-01T00:00:00")
end_date = np.datetime64("2020-01-01T00:00:00")
mask = (gauges.time >= start_date) * (gauges.time < end_date)
gauge_data_training = gauges[{"time": mask}]

In [ ]:
gauge_bins = np.concatenate([
    0.2 * np.arange(20) - 0.1 ,
    4.1 + 1.0 * np.arange(16),
    20.1 + 4.0 * np.arange(20)
])
n_gauges = gauge_data_training.gauges.size
months = gauge_data_training.time.dt.month.data.reshape(-1, 1)
months = np.tile(months, (1, n_gauges))
sp = gauge_data_training.surface_precip.data
counts_gauges, _, _ = np.histogram2d(months.ravel(), sp.ravel(), bins=(month_bins, gauge_bins))

## Precipitation distributions

In [ ]:
from hydronn.utils import adapt_gauge_precip
sp = gauges.surface_precip.data
mean = sp[sp >= 0.0].mean()
mean_adapted = adapt_gauge_precip(sp[sp >= 0.0]).mean()
(mean_adapted - mean) / mean

In [ ]:
from hydronn.utils import load_style
load_style()

In [ ]:
 seasons = {
    "DJF": [11, 0, 1],
    "MAM": [2, 3, 4],
    "JJA": [5, 6, 7],
    "SON": [8, 9, 10]
}

In [ ]:
import seaborn as sns
sns.reset_orig()
palette = sns.color_palette("twilight_shifted", 8)
colors = [palette[0],
          palette[2],
          palette[5],
          palette[6]]

In [ ]:
f, axs = plt.subplots(1, 3, figsize=(15, 5), gridspec_kw={"width_ratios": [1.0, 1.0, 0.3]})

ax = axs[0]
handles = []
x = 0.5 * (bins[1:] + bins[:-1])
for i, (name, months) in enumerate(seasons.items()):
    y = np.sum(counts[months], axis=0)
    y = y / y.sum() / np.diff(bins)
    handles += ax.plot(x, y, label=name, c=colors[i])
    
ax.set_xlim([1e-1, 1e2])
ax.set_ylim([1e-5, 1e-1])
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlabel("Rain rate [$\si{\milli \meter \per \hour}$]")
ax.set_ylabel("Probability density [$(\si{\milli \meter \per \hour})^{-1}$]")
ax.set_title("(a) Training data", loc="left")
    
ax = axs[1]

x = 0.5 * (bins[1:] + bins[:-1])
for i, (name, months) in enumerate(seasons.items()):
    y = np.sum(counts[months], axis=0)
    y = y / y.sum() / np.diff(bins)
    handles += ax.plot(x, y, label=name, c="grey", alpha=0.6)

handles = []
x = 0.5 * (gauge_bins[1:] + gauge_bins[:-1])
for i, (name, months) in enumerate(seasons.items()):
    y = np.sum(counts_gauges[months], axis=0)
    y = y / y.sum() / np.diff(gauge_bins)
    handles += ax.plot(x, y, label=name, c=colors[i])

    

ax.set_xlim([1e-1, 1e2])
ax.set_ylim([1e-5, 1e-1])
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlabel("Rain rate [$\si{\milli \meter \per \hour}$]")
ax.set_yticklabels([])
ax.set_title("(b) Gauge data", loc="left")

ax = axs[2]
ax.set_axis_off()
ax.legend(handles=handles, loc="center left")

f.savefig("../plots/training_data_statistics.pdf")

## Calculate the correction

Calculate a priori distributions corresponding to the different error assumptions.

In [ ]:
import quantnn.density as qd
from hydronn.utils import adapt_gauge_precip
sp_gauges = adapt_gauge_precip(gauge_data_training.surface_precip.data)
p_a_gauges, _ = np.histogram(sp_gauges, bins=bins, density=True)

In [ ]:
import quantnn.density as qd
p_a_gpm = qd.normalize(counts.sum(0), bins)

In [ ]:
p_a_gpm_indep = p_a_gpm.copy()
bins[0] = 1e-3
for i in range(5):
    p_a_gpm_indep = qd.add(p_a_gpm_indep, bins, p_a_gpm, bins, bins)
p_a_gpm_indep *= 6.0    
# A priori for independence assumption is on different grid.
bins_indep = bins / 6.0
p_a_gpm_indep = qd.normalize(p_a_gpm_indep, bins_indep, density=True)

## Calculate correction ratios for all retrieval configurations.

In [ ]:
r_dep = p_a_gauges / p_a_gpm
r_dep[p_a_gpm < 1e-30] = 0.0
r_dep = np.minimum(r_dep, 1e3)

In [ ]:
x = 0.5 * (bins[1:] + bins[:-1])
x_indep = 0.5 * (bins_indep[1:] + bins_indep[:-1])
p_a_gpm_indep_r = np.interp(x, x_indep, p_a_gpm_indep)
r_indep = p_a_gauges / p_a_gpm_indep_r
r_indep[p_a_gpm_indep_r < 1e-30] = 0.0
r_indep = np.minimum(r_indep, 1e3)

In [ ]:
p = p_a_gauges * np.diff(bins)
p_a_gauges_4 = 0.5 * (p[1::2] + p[::2]) / np.diff(bins_4)
p = p_a_gpm * np.diff(bins)
p_a_gpm_4 = 0.5 * (p[1::2] + p[::2]) / np.diff(bins_4)
p = p_a_gpm_indep_r * np.diff(bins)
p_a_gpm_indep_4 = 0.5 * (p[1::2] + p[::2]) / np.diff(bins_4)

In [ ]:
r_dep_4 = p_a_gauges_4 / p_a_gpm_4
r_dep_4[p_a_gpm_4 < 1e-20] = 0.0
r_indep_4 = p_a_gauges_4 / p_a_gpm_indep_4
r_indep_4[p_a_gpm_indep_4 < 1e-20] = 0.0
r_indep_4 = np.minimum(r_indep_4, 1e3)

In [ ]:
r_dep_4 = p_a_gauges_4 / p_a_gpm_4
r_dep_4[p_a_gpm_4 < 1e-20] = 0.0
r_dep_4 = np.minimum(r_dep_4, 1e3)

In [ ]:
dataset = xr.Dataset({
    "ratios_dep": (("bins",), r_dep),
    "ratios_indep": (("bins",), r_indep)
})
dataset.to_netcdf("../data/correction.nc")

In [ ]:
dataset = xr.Dataset({
    "ratios_dep": (("bins",), r_dep_4),
    "ratios_indep": (("bins",), r_indep_4)
})
dataset.to_netcdf("../data/correction_4.nc")

## Plot correction

In [ ]:
f, axs = plt.subplots(1, 2, figsize=(10, 5))
x = 0.5 * (bins[1:] + bins[:-1])
x_indep = 0.5 * (bins_indep[1:] + bins_indep[:-1])
x_4 = 0.5 * (bins_4[1:] + bins_4[:-1])

ax = axs[0]
ax.plot(x, p_a_gauges, c="dimgrey", ls="--", label="Gauges")
ax.plot(x, p_a_gpm, c="C0", label="Dep.")
ax.plot(x_indep, p_a_gpm_indep, c="C1", label="Indep.")
ax.set_xlim([1e-3, 1e2])
ax.set_ylim([1e-6, 1e3])
ax.set_ylabel("PDF [$(\si{\milli \meter \per \hour})^{-1}$]")
ax.set_xlabel("Precipitation rate [$\si{\milli \meter \per \hour}$]")
ax.set_title("(a) A priori distribution", loc="left")
ax.legend()

ax.set_xscale("log")
ax.set_yscale("log")

ax = axs[1]
ax.plot(x, r_dep)
ax.plot(x, r_indep)
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_ylim([1e-1, 1e3])
ax.set_ylabel("Correction factor")
ax.set_xlabel("Precipitation rate [$\si{\milli \meter \per \hour}$]")
ax.set_title("(b) Correction factors", loc="left")

plt.tight_layout()

f.savefig("../plots/correction_factors.pdf")